<a href="https://colab.research.google.com/github/andyarnell/sepal_mgci/blob/master/Calculation_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### **SDG 15.4.2 Subcomponent A: Calculate Global Default Values**

* This script allows batch processing for this indicator for all countries.

* Output is a combined excel file on your Google Drive.

* Runs on the cloud using [Google Colab](https://research.google.com/colaboratory/faq.html)

* Requires: [Google Earth Engine](https://earthengine.google.com/) (GEE) account and project and access to Google Drive



##Set parameters


Inputs

In [ ]:
# Google Earth Engine project

gee_project_name = "ee-andyarnellgee"  # a registered cloud project (if unsure of name see pic here: https://developers.google.com/earth-engine/cloud/assets)


# Admin boundaries asset

admin_asset_id = "FAO/GAUL/2015/level0" # administrative units feature collection

admin_asset_property_name = "ADM0_NAME" # property/column name for selecting admin boundaries (e.g. ISO3 code or country name)


# Land cover assets

# For SUB_A indicator, we need to set the following structure.
a_years = {
    1: {"asset": "users/amitghosh/sdg_module/esa/cci_landcover/2000", "year": 2000}, # baseline
    2: {"year": 2003, "asset": "users/amitghosh/sdg_module/esa/cci_landcover/2003"}, # subsequent reporting years...
    3: {"year": 2007, "asset": "users/amitghosh/sdg_module/esa/cci_landcover/2007"},
    4: {"year": 2010, "asset": "users/amitghosh/sdg_module/esa/cci_landcover/2010"},
}


Outputs

---



In [ ]:
final_report_folder = "sdg_15_4_2_A_combined_report" # folder name in Google Drive for final output (if doesnt exist creates one)

final_report_name = "sdg_15_4_2_A_default_global.xlsx" # file name for final excel output

# export GEE tasks or not
export = False # default: True. Set to False if debugging or limiting accidental re-exporting of tasks

# prints more messages
debug = False # default: False. Set to True if debugging code

Temporary outputs


In [ ]:
stats_csv_folder = "sdg_15_4_2_A_csvs" # for storing stats tables exported from GEE for each admin boundary/AOI

excel_reports_folder = "sdg_15_4_2_A_reports" # for storing formatted excel tables for each admin boundary/AOI

error_log_file_path = drive_home + excel_reports_folder + "/"+"1_error_log" +".csv" # for storing errors


### Setup Google Earth Engine
Launches access request pop up window

In [ ]:
import ee # google earth engine

ee.Authenticate()

ee.Initialize(project=gee_project_name) # NB gee project name is defined in parameters section

#### Access GitHub
Clones repository for SDG 15.4.2 into colab. Allows functions, lookup tables etc


In [3]:
# Change the current working directory to "/content".
%cd "/content"

# Clone the GitHub repository "sepal_mgci" into the current directory.
# NB if 'fatal' error on reruns, it's just saying it already exists
!git clone https://github.com/sepal_mgci.git

/content
Cloning into 'sepal_mgci'...
remote: Not Found
fatal: repository 'https://github.com/sepal_mgci.git/' not found


#### Install required packages

In [4]:
# Load the autoreload extension to automatically reload modules.
%load_ext autoreload

# Set autoreload to reload all modules before executing code.
%autoreload 2

# Function to install a package if it's not already installed
def install_if_not_exists(package_name):
    try:
        __import__(package_name)
        print(f"{package_name} is already installed.")
    except ImportError:
        !pip install -q {package_name}
        print(f"{package_name} has been installed.")

# List of packages to install if not already installed
packages_to_install = ['geemap', 'unidecode', 'google-api-python-client',
                      'google-auth-httplib2', 'google-auth-oauthlib','sepal_ui']

# Install necessary packages
for package in packages_to_install:
    install_if_not_exists(package)

geemap is already installed.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.5/235.5 kB 1.8 MB/s eta 0:00:00
unidecode has been installed.
google-api-python-client has been installed.
google-auth-httplib2 has been installed.
google-auth-oauthlib has been installed.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 893.2/893.2 kB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.1/6.1 MB 23.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.7/53.7 kB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 80.4/80.4 kB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.1/17.1 MB 21.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 553.7/553.7 kB 24.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 20.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 289.9/289.9 kB 19.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 19.1 MB/s eta 0:00:00
     ━

#### Import packages

In [ ]:
import os
from datetime import datetime # for time stamping error log
import pandas as pd # pandas library for tabular data manipulation
import re # for manipulating strings
from unidecode import unidecode # converting symbols in country names to ascii compliant (required for naming GEE tasks)

# for accessing google drive
from google.colab import auth, drive
from googleapiclient.discovery import build

# formatting excel report file
from openpyxl.utils import get_column_letter
from openpyxl.styles import Alignment

# Change current directory to sepal_mgci (i.e. the local copy of the github repository)
%cd "/content/sepal_mgci"

# Import parameters for the default DEM asset and a lookup table for land cover reclassification
from component.parameter.module_parameter import DEM_DEFAULT, LC_MAP_MATRIX

# Import scripts and modules from cloned GitHub repository (i.e., functions for indicator calculation and formatting)
from component.scripts.gee import reduce_regions # for running summary statistics in GEE
from component.scripts.scripts import get_a_years, map_matrix_to_dict, parse_result, read_from_csv# parameter prep and reformatting
from component.scripts import sub_a, sub_b, mountain_area as mntn ###TO DO: ADD DESCRIPTIONS

print("Imports complete")

/content/sepal_mgci
Imports complete


#### Setup Google Drive access
Launches access request pop up window

In [2]:
drive.mount('/content/drive')

drive_home ="/content/drive/MyDrive/" # Google Drive location. Typically this is a standard path, unless you manually changed it


NameError: name 'drive' is not defined

#### Functions
(TO DO: store functions somewhere else in GitHub - if Daniel is happy - and import as a bit of a distraction)

In [ ]:
def folder_exists(folder_name, parent_folder_id=None):
    """
    Check if a folder exists in Google Drive.

    Args:
    - folder_name (str): Name of the folder to check.
    - parent_folder_id (str): ID of the parent folder where to search for the folder.
                              Default is None, meaning the search will be performed in the root.

    Returns:
    - bool: True if the folder exists, False otherwise.
    """
    # Authenticate user
    auth.authenticate_user()

    # Build the Drive v3 service
    drive_service = build('drive', 'v3')

    # Prepare query to check if folder exists
    query = f"name='{folder_name}' and mimeType='application/vnd.google-apps.folder' and trashed=false"
    if parent_folder_id:
        query += f" and '{parent_folder_id}' in parents"

    try:
        # Execute the search query
        folders = drive_service.files().list(q=query, fields='files(id)', includeItemsFromAllDrives=True, supportsAllDrives=True).execute().get('files', [])
        return bool(folders)
    except Exception as e:
        print(f"An error occurred: {e}")
        return False


def create_folder(folder_name, parent_folder_id=None):
    """
    Create a folder in Google Drive.

    Args:
    - folder_name (str): Name of the folder to be created.
    - parent_folder_id (str): ID of the parent folder where the new folder will be created.
                              Default is None, meaning the folder will be created in the root.

    Returns:
    - str: ID of the newly created folder.
    """
    # Authenticate user
    auth.authenticate_user()

    # Build the Drive v3 service
    drive_service = build('drive', 'v3')

    # Prepare folder metadata
    folder_metadata = {
        'name': folder_name,
        'mimeType': 'application/vnd.google-apps.folder'
    }
    if parent_folder_id:
        folder_metadata['parents'] = [parent_folder_id]

    # Create the folder
    folder = drive_service.files().create(body=folder_metadata, fields='id').execute()

    # Return the ID of the newly created folder
    return folder.get('id')


def create_folder_if_not_exists(folder_name, parent_folder_id=None):
    """
    Create a folder in Google Drive if it doesn't already exist.

    Args:
    - folder_name (str): Name of the folder to be created.
    - parent_folder_id (str): ID of the parent folder where the new folder will be created.
                              Default is None, meaning the folder will be created in the root.

    Returns:
    - str: ID of the newly created folder or the existing folder if it already exists.
    """
    if folder_exists(folder_name, parent_folder_id):
        print(f"Folder '{folder_name}' already exists.")
        return None
    else:
        return create_folder(folder_name, parent_folder_id)


def sanitize_description(description):
    allowed_characters_pattern = r"[^a-zA-Z0-9.,:;_ \-]"  # Define a regex pattern for characters not in the allowed set
    sanitized_description = re.sub(allowed_characters_pattern, "", description)  # Remove characters not in the allowed set
    return sanitized_description

def append_excel_files(file_paths, num_sheets, output_file_path):
    # Initialize a dictionary to store combined DataFrames from different files
    combined_dfs = {}

    # Initialize a counter to track the progress of file processing
    counter = 0

    # Iterate over each file path in the list
    for file_path in file_paths:
        # Load the Excel file
        xls = pd.ExcelFile(file_path)  # Reads file and stores as an ExcelFile object (using the Pandas library)

        # Increment the counter for each iteration
        counter += 1

        # Read each sheet from the Excel file into a DataFrame
        # Only read up to num_sheets specified
        dfs = {sheet_name: xls.parse(sheet_name) for sheet_name in xls.sheet_names[:num_sheets]}

        # Append the DataFrames to the combined_dfs dictionary
        for sheet_name, df in dfs.items():
            if sheet_name in combined_dfs:
                # If the sheet already exists in combined_dfs, concatenate the current DataFrame with the existing one
                combined_dfs[sheet_name] = pd.concat([combined_dfs[sheet_name], df], ignore_index=True)
            else:
                # If the sheet does not exist in combined_dfs, add the DataFrame directly
                combined_dfs[sheet_name] = df

        # Print the progress of processing, overwriting the previous progress
        print(f"\rProcessing {counter}/{len(file_paths)}: {file_path}", end="")

    # Write the combined DataFrames to the specified output file path
    with pd.ExcelWriter(output_file_path) as writer:
        for sheet_name, df in combined_dfs.items():
            # Write each DataFrame to a separate sheet in the output Excel file
            df.to_excel(writer, sheet_name=sheet_name, index=False)

#### Building output folders

In [ ]:
reports_combined_file_path = os.path.join(drive_home, final_report_folder, final_report_name) # File path for the combined final report

# Creating folders in Google Drive (if don't alreaady exist)
create_folder_if_not_exists(stats_csv_folder)
create_folder_if_not_exists(excel_reports_folder)
create_folder_if_not_exists(final_report_folder)

Creating list of boundaries to process

In [ ]:
# admin boundary
admin_boundaries = ee.FeatureCollection(admin_asset_id)

# list to process
list_of_countries = admin_boundaries.aggregate_array(admin_asset_property_name).getInfo()

list_of_countries = list(set(list_of_countries)) # remove dupicates
print ("Length of admin boundaries to process", len(list_of_countries))
print ("Length of distinct admin boundaries to process", (len(set(list_of_countries))))


Length of admin boundaries to process 276
Length of distinct admin boundaries to process 276


## SUB INDICATOR A

Read the default land cover remapping table and convert it to a dictionary

In [ ]:
default_map_matrix = map_matrix_to_dict(LC_MAP_MATRIX)

Select years of land cover to process

In [ ]:
# extracts the years from the a_years dictionary (as defined in parameters)
single_years = [y["year"] for  y in a_years.values()]

#### Run area statistics within admin boundaries
* Runs for each country and each mountain biobelt
* Gets area of land cover reclassified into the 10 SEAM classes
* Repeat for each year specified


In [ ]:
# you can monitor your GEE tasks here : https://code.earthengine.google.com/tasks (and you can bulk cancel all if needed!)

counter=0 # starting place of counter used to keep track of number of tasks that are being run

for aoi_name in list_of_countries:

    aoi = admin_boundaries.filter(ee.Filter.eq(admin_asset_property_name,aoi_name))#.first()

    # gets areas of landcover in each mountain belt in each country
    # uses reduce_regions function imported from the cloned sepal_mgci git hub repository (see Imports section)
    # pixels counted at native resolution (scale) of input land cover (or DEM if RSA implementation)
    process = ee.FeatureCollection([
        ee.Feature(
            None,
            reduce_regions(
                aoi,
                remap_matrix=default_map_matrix,
                rsa=False,
                # dem=param.DEM_DEFAULT,
                dem=DEM_DEFAULT, #default digital elevation model (DEM). Relevant for the real surface area (RSA) implementation.
                lc_years= year,
                transition_matrix=False
            )
        ).set("process_id", year[0]["year"])
        for year in get_a_years(a_years) # creates GEE images and runs stats on each. Images to run are in the 'a_years" dictionary (above)
    ])

    #make name acceptable for running tasks (i.e., removes special characters)
    task_name = str(sanitize_description(unidecode(aoi_name)))


    task = ee.batch.Export.table.toDrive(
        **{  #asterisks unpack dictionary into keyword arguments format
            "collection": process,
            "description": task_name,
            "fileFormat": "CSV",
            "folder":stats_csv_folder,
            "selectors": [
                "process_id",
                "sub_a",
            ],
        }
    )

    counter+=1

    print (f"\r process {counter}/{len(list_of_countries)} {aoi_name} ", end="") #print in place (remove \r and end="" for verbose version)

    if export:
      task.start()



 process 276/276 Colombia 

# Read, process, and create report tables

#####Manually check your earth engine task status, once the tasks are complete, run the next cell. https://code.earthengine.google.com/tasks

This formats individual excel reports for each country.
See Error_log.csv for missing files/errors

In [ ]:
# Initialize the counter
counter = 0

# Loop over each AOI name in the list of countries
for aoi_name in list_of_countries:
    counter += 1

    # Clean the AOI name
    aoi_name_clean = str(sanitize_description(unidecode(aoi_name)))

    # Construct the file path for the stats CSV file
    stats_csv_file = aoi_name_clean + ".csv"
    stats_csv_file_path = os.path.join(drive_home, stats_csv_folder, stats_csv_file)

    message = f"Process {counter}, {stats_csv_file}"

    try:
        # Read the results from the CSV file and parse it to a dictionary
        dict_results = read_from_csv(stats_csv_file_path)

        details = {
            "geo_area_name": aoi_name,
            "ref_area": " ",
            "source_detail": " ",
        }

        # Generate reports for the sub_a and mtn indicators
        sub_a_reports = [sub_a.get_reports(parse_result(dict_results[year]["sub_a"], single=True), year, **details) for year in single_years]
        mtn_reports = [mntn.get_report(parse_result(dict_results[year]["sub_a"], single=True), year, **details) for year in single_years]

        # Concatenate the mtn reports
        mtn_reports_df = pd.concat(mtn_reports)

        # Concatenate the sub a reports
        er_mtn_grnvi_df = pd.concat([report[0] for report in sub_a_reports])
        er_mtn_grncov_df = pd.concat([report[1] for report in sub_a_reports])

        # Define the output report file path
        report_file_path = os.path.join(drive_home, excel_reports_folder, aoi_name_clean + ".xlsx")

        # Create the Excel file with the reports
        with pd.ExcelWriter(report_file_path) as writer:
            mtn_reports_df.to_excel(writer, sheet_name="Table1_ER_MTN_TOTL", index=False)
            er_mtn_grncov_df.to_excel(writer, sheet_name="Table2_ER_MTN_GRNCOV", index=False)
            er_mtn_grnvi_df.to_excel(writer, sheet_name="Table3_ER_MTN_GRNCVI", index=False)

            # Adjust column widths and alignment for each sheet
            for sheetname in writer.sheets:
                worksheet = writer.sheets[sheetname]
                for col in worksheet.columns:
                    max_length = max(len(str(cell.value)) for cell in col)
                    column = col[0]
                    adjusted_width = max(max_length, len(str(column.value))) + 4
                    worksheet.column_dimensions[get_column_letter(column.column)].width = adjusted_width

                    # Align "obs_value" column to the right
                    if "OBS" in column.value:
                        for cell in col:
                            cell.alignment = Alignment(horizontal="right")

    except Exception as e:
        # If an error occurs, catch the exception and handle it
        message = f"process {counter}, {stats_csv_file}, Error: {e}"

        # Get the current time
        current_time = datetime.now().strftime('%Y-%m-%d %H:%M:%S')

        # Write the error message and file name to the error log file
        error_info = pd.DataFrame([[stats_csv_file, str(e), current_time]], columns=['File Name', 'Error Message', 'Time'])

        mode = 'w' if not os.path.exists(error_log_file_path) else 'a'
        header = False if os.path.exists(error_log_file_path) else True

        # Append or write to the error log file
        error_info.to_csv(error_log_file_path, mode=mode, header=header, index=False)

    print(message)


Process 1, New Zealand.csv
process 2, Tokelau.csv, Error: [Errno 2] No such file or directory: '/content/drive/MyDrive/sdg_15_4_2_A_csvs/Tokelau.csv'
Process 3, Lao Peoples Democratic Republic.csv
process 4, British Virgin Islands.csv, Error: [Errno 2] No such file or directory: '/content/drive/MyDrive/sdg_15_4_2_A_csvs/British Virgin Islands.csv'
Process 5, Kyrgyzstan.csv
process 6, Falkland Islands Malvinas.csv, Error: [Errno 2] No such file or directory: '/content/drive/MyDrive/sdg_15_4_2_A_csvs/Falkland Islands Malvinas.csv'
Process 7, Republic of Korea.csv
Process 8, Seychelles.csv
Process 9, Liberia.csv


/content/sepal_mgci/component/scripts/sub_a.py:135: RuntimeWarning: invalid value encountered in scalar divide
  green_non_green.sum()["sumgreen"]
/content/sepal_mgci/component/scripts/sub_a.py:135: RuntimeWarning: invalid value encountered in scalar divide
  green_non_green.sum()["sumgreen"]
/content/sepal_mgci/component/scripts/sub_a.py:135: RuntimeWarning: invalid value encountered in scalar divide
  green_non_green.sum()["sumgreen"]
/content/sepal_mgci/component/scripts/sub_a.py:135: RuntimeWarning: invalid value encountered in scalar divide
  green_non_green.sum()["sumgreen"]


Process 10, Maldives.csv
process 11, Greenland.csv, Error: [Errno 2] No such file or directory: '/content/drive/MyDrive/sdg_15_4_2_A_csvs/Greenland.csv'


/content/sepal_mgci/component/scripts/sub_a.py:135: RuntimeWarning: invalid value encountered in scalar divide
  green_non_green.sum()["sumgreen"]
/content/sepal_mgci/component/scripts/sub_a.py:135: RuntimeWarning: invalid value encountered in scalar divide
  green_non_green.sum()["sumgreen"]
/content/sepal_mgci/component/scripts/sub_a.py:135: RuntimeWarning: invalid value encountered in scalar divide
  green_non_green.sum()["sumgreen"]
/content/sepal_mgci/component/scripts/sub_a.py:135: RuntimeWarning: invalid value encountered in scalar divide
  green_non_green.sum()["sumgreen"]


Process 12, Bassas da India.csv
process 13, Guadeloupe.csv, Error: [Errno 2] No such file or directory: '/content/drive/MyDrive/sdg_15_4_2_A_csvs/Guadeloupe.csv'
Process 14, Uzbekistan.csv
Process 15, Rwanda.csv
process 16, Belize.csv, Error: [Errno 2] No such file or directory: '/content/drive/MyDrive/sdg_15_4_2_A_csvs/Belize.csv'
Process 17, Niger.csv
Process 18, Mozambique.csv


...

Process 231, Malta.csv
Process 232, Chad.csv
process 233, Montserrat.csv, Error: [Errno 2] No such file or directory: '/content/drive/MyDrive/sdg_15_4_2_A_csvs/Montserrat.csv'
process 234, Nauru.csv, Error: [Errno 2] No such file or directory: '/content/drive/MyDrive/sdg_15_4_2_A_csvs/Nauru.csv'
Process 235, Japan.csv
process 236, Bird Island.csv, Error: [Errno 2] No such file or directory: '/content/drive/MyDrive/sdg_15_4_2_A_csvs/Bird Island.csv'
process 237, Madeira Islands.csv, Error: [Errno 2] No such file or directory: '/content/drive/MyDrive/sdg_15_4_2_A_csvs/Madeira Islands.csv'
Process 238, Bosnia and Herzegovina.csv
Process 239, San Marino.csv


/content/sepal_mgci/component/scripts/sub_a.py:135: RuntimeWarning: invalid value encountered in scalar divide
  green_non_green.sum()["sumgreen"]
/content/sepal_mgci/component/scripts/sub_a.py:135: RuntimeWarning: invalid value encountered in scalar divide
  green_non_green.sum()["sumgreen"]
/content/sepal_mgci/component/scripts/sub_a.py:135: RuntimeWarning: invalid value encountered in scalar divide
  green_non_green.sum()["sumgreen"]
/content/sepal_mgci/component/scripts/sub_a.py:135: RuntimeWarning: invalid value encountered in scalar divide
  green_non_green.sum()["sumgreen"]


Process 240, Bahrain.csv
Process 241, Angola.csv
Process 242, Timor-Leste.csv
process 243, Saint Vincent and the Grenadines.csv, Error: [Errno 2] No such file or directory: '/content/drive/MyDrive/sdg_15_4_2_A_csvs/Saint Vincent and the Grenadines.csv'
process 244, Panama.csv, Error: [Errno 2] No such file or directory: '/content/drive/MyDrive/sdg_15_4_2_A_csvs/Panama.csv'
Process 245, United Republic of Tanzania.csv
Process 246, Switzerland.csv
process 247, Aruba.csv, Error: [Errno 2] No such file or directory: '/content/drive/MyDrive/sdg_15_4_2_A_csvs/Aruba.csv'
Process 248, Senegal.csv
Process 249, Zimbabwe.csv
process 250, Canada.csv, Error: [Errno 2] No such file or directory: '/content/drive/MyDrive/sdg_15_4_2_A_csvs/Canada.csv'
Process 251, Iraq.csv
process 252, Azores Islands.csv, Error: [Errno 2] No such file or directory: '/content/drive/MyDrive/sdg_15_4_2_A_csvs/Azores Islands.csv'
Process 253, China.csv
Process 254, Mexico.csv
process 255, Svalbard and Jan Mayen Islands.csv

/content/sepal_mgci/component/scripts/sub_a.py:135: RuntimeWarning: invalid value encountered in scalar divide
  green_non_green.sum()["sumgreen"]
/content/sepal_mgci/component/scripts/sub_a.py:135: RuntimeWarning: invalid value encountered in scalar divide
  green_non_green.sum()["sumgreen"]
/content/sepal_mgci/component/scripts/sub_a.py:135: RuntimeWarning: invalid value encountered in scalar divide
  green_non_green.sum()["sumgreen"]
/content/sepal_mgci/component/scripts/sub_a.py:135: RuntimeWarning: invalid value encountered in scalar divide
  green_non_green.sum()["sumgreen"]


Process 270, Latvia.csv
Process 271, Djibouti.csv
process 272, Gibraltar.csv, Error: [Errno 2] No such file or directory: '/content/drive/MyDrive/sdg_15_4_2_A_csvs/Gibraltar.csv'
process 273, Barbados.csv, Error: [Errno 2] No such file or directory: '/content/drive/MyDrive/sdg_15_4_2_A_csvs/Barbados.csv'
Process 274, Algeria.csv
process 275, Guyana.csv, Error: [Errno 2] No such file or directory: '/content/drive/MyDrive/sdg_15_4_2_A_csvs/Guyana.csv'
process 276, Colombia.csv, Error: [Errno 2] No such file or directory: '/content/drive/MyDrive/sdg_15_4_2_A_csvs/Colombia.csv'


#### Combine excel files

Make a list of files to combine

In [ ]:
# Directory path where Excel reports are stored
directory_path = os.path.join(drive_home, excel_reports_folder)

# List files in the directory with '.xlsx' extension
files = [file for file in os.listdir(directory_path) if file.endswith('.xlsx')]

# Create a list of full file paths
full_file_paths = [os.path.join(directory_path, file) for file in files]

# Print the number of Excel files found in the folder
print(f"Number of Excel files in folder: {len(full_file_paths)}")


Number of Excel files in folder: 166


##### Run function to combine into a single report

In [ ]:
append_excel_files(file_paths=full_file_paths,num_sheets=3,output_file_path=reports_combined_file_path)

print (f"\n Complete! Output file for SDG 15.4.2 Component A here: {reports_combined_file_path}")

Processing 166/166: /content/drive/MyDrive/sdg_15_4_2_A_reports/Cyprus.xlsx
 Complete! Output file for SDG 15.4.2 Component A here: /content/drive/MyDrive/sdg_15_4_2_A_combined_report/sdg_15_4_2_A_default_global.xlsx
